## EDA Notebook

In [ ]:
data_base_schema.png

### Exploratory Data Analysis
1. Data base exploration
2. Inital observations

#### Data base exploration
To create a CLV, model we need a customer identifier, purchase timestamp, and purchase value. The data base contains more information than we need so I'm only going to open these three files for further exploration. 
- olist_customers_dataset
    - This dataset has information about the customer and its location. Use it to identify unique customers in the orders dataset.
- olist_order_items_dataset
    - d
- olist_order_payments_dataset
    - This dataset includes data about the orders payment options.

In [10]:
#import zipfile to deal with zipped data
from zipfile import ZipFile
# Create iterate over our files in the zip file and print their names
print("Data base files")
print(" ")
with ZipFile('data/customer_data_base.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   # Iterate over the list of file names in given list & print them
   for elem in listOfiles:
       print(elem)

Data base files
 
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_orders_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [14]:
#Inital imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lifetimes
from zipfile import ZipFile
import random

In [15]:
#Load in our data
# open zipped dataset and save to df's
with ZipFile("data/customer_data_base.zip") as z:
   # open the orders csv file in the dataset
   with z.open("olist_orders_dataset.csv") as a:    
      # read the dataset as dataframe "df"
      orders_df = pd.read_csv(a)
        
   # open the customers csv file in the dataset
   with z.open("olist_customers_dataset.csv") as b:    
      # read the dataset as dataframe "df"
      customers_df = pd.read_csv(b)     
        
   # open the customers csv file in the dataset
   with z.open("olist_order_payments_dataset.csv") as c:    
      # read the dataset as dataframe "df"
      payments_df = pd.read_csv(c)     


![Data_base_schema](visuals/data_base_schema.png)

#### Initial Observations

#### customers_dataset
- All we need from this dataset is `customer_unique_id`
- No null or missing values
- We have 96,096 unique customers
- No null or missing values
- All 27 Brazilian states + federal district (FD) represented
- customer_unique_id represents the unique customer identifier
- customer_id is a key to the orders dataset and represents a unique order id
    - From the data providers, "At our system each order is assigned to a unique customerid."

#### olist_order_items_dataset
- 

##### customers_dataset

In [50]:
#99441 observations
customers_df.shape

(99441, 5)

In [83]:
#We have no null or na values
customers_df.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [80]:
#We have mostly strings and an integer for zip code
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [20]:
#Explore head of dataframe
#There is a difference between customer_id and customer_unique_id, let's explore further
customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [39]:
#As the name suggests, unique_customer_id is a more unique customer identifier
print(f'Total unique customer_id: {len(customers_df["customer_id"].unique())}')
print(f'Total unique customer_unique_id: {len(customers_df["customer_unique_id"].unique())}')

Total unique customer_id: 99441
Total unique customer_unique_id: 96096


In [49]:
#We have 1.03 unique customer_id per every customer_unique_id
len(customers_df["customer_id"].unique())/len(customers_df["customer_unique_id"].unique())

1.0348089410589412

In [57]:
#All 27 Brazilian states + federal district (FD) represented
len(customers_df["customer_state"].unique())

27

In [54]:
customers_df["customer_state"].value_counts()

SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: customer_state, dtype: int64

##### olist_order_items_dataset

In [61]:
#99441 observations, same as our customers dataset
orders_df.shape

(99441, 8)

In [123]:
#Explore head of dataframe
#From this dataset we have the required elements for a CLV model; customer identifier, purchase timestamp.
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [124]:
#We have some order statuses that are concerning for a CLV model; canceled and unavailable
#We have to check if these order types are associated with zero or negative monetary values when we join datasets 
orders_df["order_status"].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [125]:
#We have have some null values to deal with in order_approved_at, order_delivered_carrier_date, order_delivered_customer_date
orders_df.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [121]:
#Dealing with order_approved_at null values first
order_approved_at_null_df = orders_df[orders_df["order_approved_at"].isnull() == True]
order_approved_at_null_df["order_status"].value_counts()

canceled     141
delivered     14
created        5
Name: order_status, dtype: int64

In [112]:
null_orders_df=orders_df[orders_df.isnull().any(axis=1)]
null_orders_df["order_status"].value_counts()

shipped        1107
canceled        619
unavailable     609
invoiced        314
processing      301
delivered        23
created           5
approved          2
Name: order_status, dtype: int64

In [85]:
#Let's check if those null values from order_approved_at, order_delivered_carrier_date, and order_delivered_customer_date
#order_delivered_customer_date are because of these troublesome order statuses

In [97]:
trouble_order_statuses = ["canceled","unavailable","invoiced","processing","created","approved"]

In [96]:
trouble_order_statuses_df = orders_df[orders_df["order_status"].isin(trouble_order_statuses)]
trouble_order_statuses_df.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 146
order_delivered_carrier_date     1781
order_delivered_customer_date    1850
order_estimated_delivery_date       0
dtype: int64

#### Data Cleaning

#### customers_dataset
- 

#### olist_order_items_dataset
- Check and remove null and missing values